## Azure Resource Graph Query with Service Principal Authentication

This Databricks notebook authenticates with Azure using a Service Principal and runs a Kusto query
against the Azure Resource Graph to retrieve resources of type "microsoft.databricks/workspaces".

**Prerequisites**:
- Azure Service Principal credentials (Tenant ID, Client ID, and Client Secret)
- Required Python packages: 
    - azure-identity (for service principal authentication)
    - azure-mgmt-resourcegraph (for interacting with Azure Resource Graph)
    - azure-mgmt-resource (for managing resources)

In [0]:
!pip install azure-identity azure-mgmt-resource azure-mgmt-resourcegraph


In [0]:
import azure.identity
print("azure-identity is installed correctly")


In [0]:
import json
from azure.identity import ClientSecretCredential
from azure.mgmt.resourcegraph import ResourceGraphClient
from azure.mgmt.resourcegraph.models import QueryRequest

# Function to authenticate using Service Principal credentials
def authenticate_with_service_principal(tenant_id, client_id, client_secret):
    credential = ClientSecretCredential(tenant_id, client_id, client_secret)
    print("Authentication successful with Service Principal.")
    return credential

# Function to run the Kusto query using Azure SDK
def run_kusto_query(credential):
    client = ResourceGraphClient(credential)

    query = 'resources | where type =~ "microsoft.databricks/workspaces"'
    query_request = QueryRequest(query=query)

    result = client.resources(query_request)
    
    print("Query executed successfully.")
    result_json = json.dumps(result.data, default=str)
    print(result_json)

# Main execution flow
if __name__ == "__main__":
    tenant_id = dbutils.secrets.get(scope="adb_trial_workspace", key="azure-tenant-id")
    client_id = dbutils.secrets.get(scope="adb_trial_workspace", key="spn-az-graph-app-id")
    client_secret = dbutils.secrets.get(scope="adb_trial_workspace", key="spn-az-graph-secret")

    credential = authenticate_with_service_principal(tenant_id, client_id, client_secret)
    run_kusto_query(credential)